In [ ]:
#| default_exp logger_loguru

In [ ]:
#| export
from __future__ import annotations

# logger_loguru
> Loguru configuration helpers for structured logging with indentation

Provides pre-configured logging setup using Loguru with notebook-friendly defaults and hierarchical indentation support for tracking nested operations.

**Key features:**  
`config_logger()` — One-line setup returning configured logger + tree logger  
`tree_logger` — Hierarchical logging with automatic indentation (`llogger.push`, `llogger.pop`)  
`LogFormatter` — Customizable format with indentation support  
`LogLevelFilter` — Dynamic level filtering with temporary override  

**When to use:** Need readable, structured logging in notebooks or scripts with hierarchical context tracking (e.g., nested async operations, recursive algorithms).

**Typical usage:**
```python
from loguru import logger
from pote.logger_loguru import config_logger

logger, llogger = config_logger(logger)
llogger.info("root")
with llogger.inc_indent():
    llogger.info("nested")
```

<!-- # Prologue -->

In [ ]:
#| export
try:
    import loguru
except ImportError as e:
    raise ImportError(
        "Logger utilities require optional dependency. "
        "Install with: pip install pote[logging]"
    ) from e

In [ ]:
#| export
import functools
import sys
from contextlib import contextmanager
from typing import Callable

import fastcore.all as F


In [ ]:
#| hide
from fastcore.test import *
from nbdev.showdoc import *  # type: ignore

In [ ]:
#| exporti
from loguru import logger
from loguru._logger import Logger


# loguru basics

Quick exploration of loguru's core features and rationale for this module's configuration choices.

Default handler:  
  - logs are emitted to `sys.stderr` by default  
  - messages can be logged with different severity levels  
  - messages are formatted using curly braces (it uses str.format() under the hood)  

In [ ]:
logger.debug("Hello, world! {}", 'aaaa')

2025-11-13 13:34:07.404 | DEBUG    | __main__:<module>:1 - Hello, world! aaaa


In notebooks, traces are not good:

In [ ]:
@logger.catch
def f(x): 100 / x  # type: ignore

def g():
    f(10)
    f(0)

g()

2025-11-13 13:34:07.408 | ERROR    | __main__:g:6 - An error has been caught in function 'g', process 'MainProcess' (43630), thread 'MainThread' (8320000192):
Traceback (most recent call last):

  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/vic/miniforge3/envs/dev/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
    │   └ <bound method Application.launch_instance of <class 'ipykernel.kernelapp.IPKernelApp'>>
    └ <module 'ipykernel.kernelapp' from '/Users/vic/miniforge3/envs/dev/lib/python3.12/site-packages/ipykernel/kernelapp.py'>
  File "/Users/vic/miniforge3/envs/dev/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
    │   └ <function IPKernelApp.start>
    └ <ipykernel.kernelapp.IPKernelApp object>
  File "/Users/vic/miniforge3/envs/dev/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 

[filter traceback in Jupyter when using @logger.catch](https://github.com/Delgan/loguru/issues/437)

In notebooks contexts, configure your handler with backtrace option disabled:

In [ ]:
logger.remove()
logger.add(sys.stderr, backtrace=False)

1

In [ ]:
g()

2025-11-13 13:34:07.429 | ERROR    | __main__:g:6 - An error has been caught in function 'g', process 'MainProcess' (43630), thread 'MainThread' (8320000192):
Traceback (most recent call last):

  File "/var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_43630/4019600944.py", line 6, in g
    f(0)
    └ <function f>

  File "/var/folders/np/k2wj6f4s3rj0m9n0yt8pkk680000gn/T/ipykernel_43630/4019600944.py", line 2, in f
    def f(x): 100 / x  # type: ignore
        │ │         └ 0
        │ └ 0
        └ <function f>

ZeroDivisionError: division by zero


Add some colors and formatting to the output:

In [ ]:
logger.remove()
i = logger.add(sys.stderr, colorize=True, format="[<fg #66a3ff>{time:YYYY-MM-DD HH:mm:ss}</fg #66a3ff>] [<fg #00ff00>{level}</fg #00ff00>] {message}")
logger.info("test {}", i)
logger.debug("Hello, world!")

[2025-11-13 13:34:07] [INFO] test 2
[2025-11-13 13:34:07] [DEBUG] Hello, world!


# formatter

Custom formatter that adds indentation tracking for hierarchical log output.

In [ ]:
#| export
class LogFormatter:
    "Formats log records with indentation for tree-structured output"
    _ind_level = 0
    
    def __init__(self) -> None:
        self.padding = 0
        self.indent = '  ' * self._ind_level
        self.fmt = "<g>{level: >7}</> | <d>{extra[indent]}</d><level>{message}</level>\n"

    def format(self, record: dict) -> str:
        "Format record with current indentation level"
        extra = record["extra"]
        if not extra.get('indent', False):  extra['indent'] = ''
        if self.padding:
            length = len("{name}:{function}:{line}".format(**record))
            self.padding = max(self.padding, length)
            record["extra"]["padding"] = " " * (self.padding - length)
        return self.fmt

_formatter = LogFormatter()

In [ ]:
fmt = LogFormatter()
test_eq(fmt.indent, '')
test_eq(LogFormatter._ind_level, 0)

record = {"extra": {}, "name": "test", "function": "fn", "line": 10}
fmt.format(record)
test_eq(record["extra"]["indent"], '')
record

{'extra': {'indent': ''}, 'name': 'test', 'function': 'fn', 'line': 10}

In [ ]:
LogFormatter._ind_level = 2
fmt2 = LogFormatter()
test_eq(fmt2.indent, '    ')  # 2 levels * 2 spaces

In [ ]:
LogFormatter._ind_level = 0  # reset for other tests

In [ ]:
#| export
class LogLevelFilter:
    "Filter log records by minimum level with context manager for temporary override"
    
    def __init__(self, level: int | str): self.level = level
        
    def __call__(self, record: dict) -> bool:
        "Return True if record meets minimum level threshold"
        return record["level"].no >= logger.level(self.level).no  # type: ignore
    
    @contextmanager
    def temp(self, tmp_level: int | str):
        "Temporarily override filter level within context"
        prev_level, self.level = self.level, tmp_level
        try: yield
        finally: self.level = prev_level

_log_level_filter = LogLevelFilter("INFO")

In [ ]:
#| export
log_config = {
    "handlers": [
        {
            "sink": sys.stdout,
            # "sink": sys.stderr,
            "backtrace": not F.IN_NOTEBOOK,
            "level": _log_level_filter.level,
            "filter": _log_level_filter,
            "colorize": True,
            # "format": "<g>{time:HHmmss}</>  | {level} | <level>{message}</level>"
            # "format": "<g>{level: >7}</> | <level>{message}</level>"
            "format": _formatter.format
        },
        # {"sink": "file.log", "serialize": True},
    ],
    # "extra": {"user": "someone"}
}

In [ ]:
# ⎸
# LEFT VERTICAL BOX LINE
# Unicode: U+23B8, UTF-8: E2 8E B8

# ⏐
# VERTICAL LINE EXTENSION
# Unicode: U+23D0, UTF-8: E2 8F 90

# │
# BOX DRAWINGS LIGHT VERTICAL
# Unicode: U+2502, UTF-8: E2 94 82

# loggers

Wrapper that manages indentation levels for hierarchical log output.

In [ ]:
#| export
from typing import Any


class tree_logger:
    "Logger wrapper managing hierarchical indentation for tree-structured output"
    
    def __init__(self, logger: Any, fmt: LogFormatter):
        self._fmt = fmt
        self.logger = logger
        
    @property
    def level(self) -> int:
        "Current indentation level (0=root)"
        return self._fmt._ind_level
    
    @property
    def indent(self) -> str:
        "Current indentation string using box drawing chars"
        return '\u2502 ' * self._fmt._ind_level

    def __getattr__(self, name: str) -> Any:
        "Proxy logger methods with indent bound; handle push/pop"
        if name == 'push': return self._push()
        elif name == 'pop': return self._pop()
        return getattr(self.logger.bind(indent=self.indent), name)

    def _push(self) -> Any:
        "Increase indent level, return logger at previous level"
        lg = self.logger.bind(indent=self.indent)
        self._fmt._ind_level += 1
        return lg
        
    def _pop(self) -> Any:
        "Decrease indent level, return logger at new level"
        self._fmt._ind_level -= 1
        return self.logger.bind(indent=self.indent)
    
    def reset(self) -> Any:
        "Reset indentation to root level"
        self._fmt._ind_level = 0
        return self.logger.bind(indent=self.indent)

    @contextmanager
    def inc_indent(self):
        "Context manager temporarily increasing indent by one level"
        self._fmt._ind_level += 1
        yield
        self._fmt._ind_level -= 1

    def bracket_logging(self, header_f: str, *attr_n):
        "Decorator for async methods: log entry with indent, auto-pop on exit"
        llogger = self
        def _wrapper(f: Callable):
            @functools.wraps(f)
            async def _mark_logging_inner(self, *args, **kwargs):
                header = header_f.format(*[
                    attr if type(attr) == str 
                    else getattr(self, str(attr, 'UTF-8')) 
                    for attr in attr_n
                ])
                llogger.push.info(f"{header}: >>>> {f.__name__}...")
                retv = await f(self, *args, **kwargs)
                llogger.pop
                return retv
            return _mark_logging_inner
        return _wrapper

# setup
> Configuration functions for quick logger setup in library modules

In [ ]:
#| export
def setup_logger(logger, name = '__main__') -> Logger:
    "Apply standard config (format, level filter, backtrace) to logger"
    logger.configure(**log_config)  # type: ignore
    return logger

~~Note logging is disabled after `setup_logger` if called from a module distinct to '__main__'.~~

In [ ]:
logger = setup_logger(logger)
logger.info('test')

   INFO | test


In [ ]:
#| export
def config_logger(
    logger: Logger  # Logger instance from each module
) -> tuple[Logger, tree_logger]:
    "Configure logger with colors and return (logger, tree_logger) tuple"
    logger = setup_logger(logger)
    logger = logger.opt(colors=True)
    logger.opt = functools.partial(logger.opt, colors=True)
    llogger = tree_logger(logger, _formatter)
    return logger, llogger

**Module usage pattern:**

Each module imports and configures its own logger:

```python
from loguru import logger
from pote.logger_loguru import config_logger

logger, llogger = config_logger(logger)

# Standard logging
logger.info("Processing started")

# Hierarchical logging  
llogger.info("Main operation")
with llogger.inc_indent():
    llogger.info("Sub-operation")
```

This gives each module independent logger configuration while maintaining consistent formatting.

In [ ]:
logger, llogger = config_logger(logger)
logger.info('configured')

   INFO | configured


In [ ]:
test_eq(llogger.level, 0)
test_eq(llogger.indent, '')
llogger.info('root level')

   INFO | root level


In [ ]:
_ = llogger.push
llogger.info("pushed once")
test_eq(llogger.level, 1)
test_eq(llogger.indent, '│ ')

   INFO | │ pushed once


In [ ]:
_ = llogger.push
llogger.info("pushed twice")
test_eq(llogger.level, 2)
test_eq(llogger.indent, '│ │ ')

   INFO | │ │ pushed twice


In [ ]:
_ = llogger.pop
test_eq(llogger.level, 1)

In [ ]:
_ = llogger.reset()
test_eq(llogger.level, 0)

In [ ]:
llogger.info('root')
with llogger.inc_indent():
    test_eq(llogger.level, 1)
    llogger.info('child')
    with llogger.inc_indent():
        test_eq(llogger.level, 2)
        llogger.info('grandchild')
    test_eq(llogger.level, 1)
    llogger.info('child')
test_eq(llogger.level, 0)
llogger.info('root')

   INFO | root
   INFO | │ child
   INFO | │ │ grandchild
   INFO | │ child
   INFO | root


In [ ]:
class _Test:
    a = 'a'
    b = 'b'
    
    @llogger.bracket_logging("<n>{}</> <y>{}</>", 'test', b'b')
    async def test_bracket_logging_b(self):
        llogger.info("inside test_bracket_logging_b")
        test_eq(llogger.level, 2)  # nested 2 levels deep
        
    @llogger.bracket_logging("<n>{}</> <y>{}</>", 'test', b'a')
    async def test_bracket_logging_a(self):
        llogger.info("inside test_bracket_logging_a")
        test_eq(llogger.level, 1)  # nested 1 level
        await self.test_bracket_logging_b()  # type: ignore
        test_eq(llogger.level, 1)  # back to 1 after inner returns

_t = _Test()
llogger.reset()
test_eq(llogger.level, 0)
await _t.test_bracket_logging_a()  # type: ignore
test_eq(llogger.level, 0)  # back to root

   INFO | test a: >>>> test_bracket_logging_a...
   INFO | │ inside test_bracket_logging_a
   INFO | │ test b: >>>> test_bracket_logging_b...
   INFO | │ │ inside test_bracket_logging_b


bracket_logging decorator with nested calls


<!-- # Colophon -->
----

In [ ]:
#|hide
#|eval: false

import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean

In [ ]:
#|hide
#|eval: false

if FC.IN_NOTEBOOK:
    nb_path = '02_logger_loguru.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)